<a href="https://colab.research.google.com/github/kaiesalmahmud/GAN/blob/main/GAN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [3]:
from typing import List

In [4]:
def create_binary_list_from_int(number: int) -> List[int]:
    if number < 0 or type(number) is not int:
        raise ValueError('Only Positive Integers are Allowed')
    
    return [int(x) for x in list(bin(number))[2:]]

In [5]:
create_binary_list_from_int(56)

[1, 1, 1, 0, 0, 0]

In [6]:
def generate_even_data(max_int, batch_size=16):
    max_length = int(math.log(max_int, 2))

    sampled_integers = np.random.randint(0, int(max_int/2), batch_size)

    labels = [1] * batch_size

    data = [create_binary_list_from_int(int(x*2)) for x in sampled_integers]
    data = [([0] * (max_length - len(x))) + x for x in data]

    return labels, data

In [7]:
labels, data = generate_even_data(128)

In [8]:
class Generator(nn.Module):
    
    def __init__(self, input_length):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length))
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
        return self.activation(self.dense_layer(x))

In [13]:
class Discriminator(nn.Module):
    def __init__(self, input_length):
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense(x))

In [18]:
def train(max_int = 128, batch_size = 16, training_steps = 500):
    input_length = int(math.log(max_int, 2))

    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    generator_optimizer = optim.Adam(generator.parameters(), lr=0.001)
    discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)

    loss = nn.BCELoss()

    for i in range(training_steps):
        generator_optimizer.zero_grad()

        noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
        generated_data = generator(noise)

        true_labels, true_data = generate_even_data(max_int, batch_size=batch_size)
        true_labels = torch.tensor(true_labels).float()
        true_data = torch.tensor(true_data).float()

        generator_discriminator_out = discriminator(generated_data)
        generator_loss = loss(generator_discriminator_out, true_labels)
        generator_loss.backward()
        generator_optimizer.step()

        discriminator_optimizer.zero_grad()
        true_discriminator_out = discriminator(true_data)
        true_discriminator_loss = loss(true_discriminator_out, true_labels)

        generator_discriminator_out = discriminator(generated_data.detach())
        generator_discriminator_loss = loss(generator_discriminator_out, torch.zeros(batch_size))
        discriminator_loss = (true_discriminator_loss + generator_discriminator_loss) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()

In [20]:
#train()